In [8]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import pickle as pickle
ratings_data = pd.read_csv("ratings.csv")
print(ratings_data.head())

   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931


In [2]:
movie_data = ratings_data.drop('timestamp', axis=1)
# movie_data = pd.merge(ratings_data, movie_names, on='movieId')
movie_data.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [4]:
users = pd.Series(pd.unique(movie_data['userId']))
print(users.size)

610


In [5]:
users_by_film = {}
for movie_id in tqdm(pd.unique(movie_data['movieId'])):
    users_by_film[movie_id]=movie_data.loc[movie_data['movieId'] == movie_id,'userId']

100%|████████████████████████████████████████████████████████████████████████████| 9724/9724 [00:03<00:00, 2894.03it/s]


In [6]:
movie_data_by_film = {}
for movie_id in tqdm(users_by_film):
    movie_data_by_film[movie_id] = movie_data.loc[(movie_data['movieId'] == movie_id),['userId','rating']]

100%|████████████████████████████████████████████████████████████████████████████| 9724/9724 [00:07<00:00, 1241.09it/s]


In [9]:
with open('users_by_film_dictionary.pkl', 'wb') as f:
    pickle.dump(users_by_film, f)
        
with open('movie_data_by_film.pkl', 'wb') as f:
    pickle.dump(movie_data_by_film, f)

In [10]:
with open('users_by_film_dictionary.pkl', 'rb') as f:
    users_by_film = pickle.load(f)
    
with open('movie_data_by_film.pkl', 'rb') as f:
    movie_data_by_film = pickle.load(f)

In [14]:
intersect_users_dict = {}
movies_ids = list(users_by_film.keys())
for i in tqdm(range(len(movies_ids))):
    movie_id_1 = movies_ids[i]
    users_1 = users_by_film[movie_id_1]
    movie_1 = movie_data_by_film[movie_id_1]
    for j in range(i+1,len(movies_ids)):
        movie_id_2 = movies_ids[j]
        movie_2 = movie_data_by_film[movie_id_2]
        users_2 = users_by_film[movie_id_2]
        
        r_k_j = movie_1.loc[(movie_1['userId'].isin(users_2)),'rating'].mean()
        r_k_i = movie_2.loc[(movie_2['userId'].isin(users_1)),'rating'].mean()
        
        intersect_users_dict[(movie_id_1,movie_id_2)]=r_k_j-r_k_i

  2%|█▍                                                                           | 174/9724 [08:44<8:00:04,  3.02s/it]


KeyboardInterrupt: 

In [16]:
import asyncio
import time
def background(f):
    def wrapped(*args, **kwargs):
        return asyncio.get_event_loop().run_in_executor(None, f, *args, **kwargs)

    return wrapped

@background
def your_function(users_1,movie_1,j):
    movie_id_2 = movies_ids[j]
    movie_2 = movie_data_by_film[movie_id_2]
    users_2 = users_by_film[movie_id_2]
        
#         intersect_users = movie_2[movie_2['userId'].isin(users_1)]['userId']
#         r_k_j = movie_1[(movie_1['userId'].isin(intersect_users))]['rating'].mean()
#         r_k_i = movie_2[(movie_2['userId'].isin(intersect_users))]['rating'].mean()
        
    r_k_j = movie_1.loc[(movie_1['userId'].isin(users_2)),'rating'].mean()
    r_k_i = movie_2.loc[(movie_2['userId'].isin(users_1)),'rating'].mean()
        
    intersect_users_dict[(movie_id_1,movie_id_2)]=r_k_j-r_k_i
#     print('function finished for '+str(argument))


    
intersect_users_dict = {}
movies_ids = list(users_by_film.keys())    
for i in tqdm(range(len(movies_ids))):
    movie_id_1 = movies_ids[i]
    users_1 = users_by_film[movie_id_1]
    movie_1 = movie_data_by_film[movie_id_1]
    for j in range(i+1,len(movies_ids)):
        your_function(users_1,movie_1,j)


print('loop finished')

  1%|▉                                                                           | 124/9724 [09:01<11:38:26,  4.37s/it]


KeyboardInterrupt: 

In [ ]:
with open('intersect_users_dict.pkl', 'wb') as f:
    pickle.dump(intersect_users_dict, f)

In [ ]:
with open('movie_data_by_film.pkl', 'rb') as f:
    movie_data_by_film = pickle.load(f)

In [41]:
users_ratings_prediction = pd.DataFrame([],columns=['user_id','movie_id','rating'])
intersect_users_dict = {}
# для всех пользователей
for _, t in users.iteritems():
   
    user_movies = movie_data.loc[movie_data['userId']==t]
    user_movies = user_movies.drop_duplicates(subset=["userId", "movieId"])
    user_movies_ids = movie_data['movieId']
    not_user_movies = pd.Series(pd.unique(movie_data.loc[~movie_data['movieId'].isin(user_movies),'movieId']))
     # для всех неоценнёных пользователем фильмов
    for _, mj in  tqdm(not_user_movies.iteritems()):
        R_j=[]
        dev_j_i_sum=0

#         users_mj = users_by_film[mj]
#         movie_mj = movie_data_by_film[mj]
    #     для всех оцененных пользователем фильмов
        for _, mi in user_movies_ids.iteritems():

            users_mi = users_by_film[mi]
#             movie_mi = movie_data_by_film[mi]

    #         ищем пользователем которые оценили оба фильма
    #         intersect_users = np.intersect1d(users_mj,users_mi)
#             intersect_users = movie_mi.loc[movie_mi['userId'].isin(users_mj),'userId']
#             index = (mj,mi) if mj < mi else (mi,mj)
#             intersect_value = intersect_users_dict.get(index,float("nan"))

    #     если такие пользователи нашлись
#             if (intersect_users.size > 0):
#             if (not pd.isna(intersect_value)):                                            
    #             movie_mj = movie_data_by_film[mj]
#                 r_k_j = movie_mj.loc[(movie_mj['userId'].isin(intersect_users)),'rating'].mean()
#                 r_k_i = movie_mi.loc[(movie_mi['userId'].isin(intersect_users)),'rating'].mean()
#                 r_k_j = movie_1.loc[(movie_1['userId'].isin(users_2)),'rating'].mean()
#                 r_k_i = movie_2.loc[(movie_2['userId'].isin(users_1)),'rating'].mean()
    #             считаем среднюю разницу между оценками этих фильмов
#                 dev_j_i = (r_k_j-r_k_i)
                
#                 dev_j_i = intersect_value if mj < mi else -intersect_value
    #     и прибавляем оценку текущего пользователя
#                 dev_j_i_sum += dev_j_i 
    #             +  movie_mi.loc[(movie_mi['userId'] == t),'rating'].item()
#                 R_j.append(mi)

    #         получаем среднюю оценку для неоценнённого фильма по оценённым фильмам пользователя
#         P_j = (user_movies.loc[user_movies['movieId'].isin(R_j),'rating'].sum() + dev_j_i_sum)/len(R_j) if len(R_j) > 0 else float("nan")
        P_j=0
#         users_ratings_prediction.loc[users_ratings_prediction.shape[0]] = [t, mj, P_j]

print(users_ratings_prediction.sort_values(['rating'], ascending=[0]))

1416it [00:43, 32.35it/s]


KeyboardInterrupt: 

In [30]:
r_k_j = movie_data.loc[(movie_data['userId'].isin([100500])),'rating'].mean()
r_k_i = movie_data.loc[(movie_data['userId'].isin([100500])),'rating'].mean()

In [26]:
r_k_j

3.9482758620689653

In [27]:
r_k_i

nan

In [28]:
r_k_j-r_k_i

nan

In [29]:
r_k_i-r_k_j

nan